In [2]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


You should consider upgrading via the 'c:\users\gjkh4\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
from itertools import count
from selenium import webdriver

import ssl

#  전체 댓글 크롤링

In [83]:
#전독시 웹툰 페이지 : https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed
import re
result = []

wd = webdriver.Chrome('C:/Users/gjkh4/CRWL_김지현/chromedriver_win32/chromedriver.exe')
origin_URL = 'https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed'
#Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=747269&no=29#'
#Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=759923&no=1#'


# 먼저 전독시 웹툰 들어가서 가장 최근 만화 클릭하기
wd.get(origin_URL)
# 휴재가 아니라면 가장 최근 웹툰이 나오기에 바로 그 페이지에서 회별 숫자를 받기 위해 title을 찾는다.
title_num=wd.find_elements_by_class_name('title')
#print(title_num[0].text) # 가장 첫 번째 회차의 이름을 받아옴.



wd.find_element_by_partial_link_text(title_num[0].text).click() # 가장 첫 번째 회차의 이름으로 해당 페이지에 접속
time.sleep(3)



#print(wd.current_url) # 현재 주소
recent_num_url = wd.current_url
# 가장 최신 회차의 주소를 알아내서 거기서 회차수만 받아옴.
no_result = re.search('no=(.+?)&', recent_num_url) # 주소에서 no = 이 부분이 회차 num인데 no=과 & 사이에 숫자만 받아옴
if no_result: #그에 대한 결과가 있다면
    found = no_result.group(1) # found에 저장함.
else : # 만약 주소에서 no이 가장 마지막에 위치한다면 결과값이 안나올 수 있기에 다시 한 번 else로 잡음
    no_result = re.search('no=(.+?)', recent_num_url)
    found = no_result.group(1)
    
int_found = int(found)# 가장 최근 회차를 int형으로 받아옴


# 최신 횟차 수 만큼 반복문을 돌려서 모든 회차의 댓글을 긁어옴,
for toon_part in range(int_found, 0, -1) :
    Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=747269&no=%s'% str(toon_part)
    wd.get(Webtoon_URL)
    wd.find_element_by_partial_link_text('전체댓글').click()        
    time.sleep(3)

    # 착한 댓글만 보게 하는 것 설정 제거
    wd.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
    time.sleep(0.4)

    while True : # 트루로 해두고
        #time.sleep(0.4)
        try:
            pages_num=wd.find_elements_by_class_name('u_cbox_num_page') # 현재 페이지에 뜨는 모든 페이지 num

            for i in range(len(pages_num)) : # 페이지 수만큼 진행
                time.sleep(0.4) #이거 안넣으면 첫페이지가 두번 크롤링됨
                #여기서 반복문으로 페이지마다 페이지 소스코드 받아외야 함.
                rcv_data = wd.page_source        
                soupData = BeautifulSoup(rcv_data, 'html.parser')

                # 현재 페이지 수 확인
                page_data=soupData.findAll('strong', attrs={'class': 'u_cbox_page'}) # 현재 눌려있는 페이지 번호만 갖는 class
                now_page=list(page_data[0].strings)[0] # page_data[0]는 하나뿐이라 무조건 0이고, 뒤에 []는 0이랑 1이 들어갈 수 있는데, 0이 해당하는 현제페이지수다.
                print(now_page)

                # 댓글 받아오기
                comment = soupData.findAll('span', attrs={'class': 'u_cbox_contents'})
                for comment_str in comment:
                    re = list(comment_str.strings)
                    result.append(re)

                #현재 페이지 번호(now_page)가 마지막 페이지 번호인지 확인하기 위해
                new_pages_num=wd.find_elements_by_class_name('u_cbox_num_page')# 창의 내용이 새로 고쳐져서 기존 객체를 계속 쓸 수 없기에 새 객체생성
                chk_lastnum =int(new_pages_num[len(new_pages_num)-1].text) # 현재페이지에 나와있는 페이지 번호 중 가장 마지막 페이지수

                if chk_lastnum > int(now_page) : #  마지막 페이지보다 현재페이지가 낮으면 계속 페이지를 넘기면 됨
                    new_pages_num[i+1].click() # 새로이 만든 객체로부터 click실행
                    time.sleep(0.3)


            temp_nextbtn=wd.find_element_by_class_name('u_cbox_next')        
            # 10,20,30 ... 페이지면 다음버튼을 누르고 아니라면 종료 & 마지막 페이지가 10의 배수일 수 있기에 다음버튼의 유무 확인
            if int(now_page) % 10 == 0 and temp_nextbtn.text != '' :
                wd.find_element_by_class_name('u_cbox_next').click()
                time.sleep(0.3)
                continue
            else :
                break   
            
        except UnexpectedAlertPresentException: # 데이터 전송에 실패했다는 문구 발생
                self.driver.switch_to.alert().accept()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


NameError: name 'UnexpectedAlertPresentException' is not defined

In [ ]:
 # 렉때문에 두 번에 걸쳐서 ==> 하나는 마지막 화부터 16화까지

In [2]:
#전독시 웹툰 페이지 : https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed
import re
result = []

wd = webdriver.Chrome('C:/Users/gjkh4/CRWL_김지현/chromedriver_win32/chromedriver.exe')
origin_URL = 'https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed'


# 먼저 전독시 웹툰 들어가서 가장 최근 만화 클릭하기
wd.get(origin_URL)
# 휴재가 아니라면 가장 최근 웹툰이 나오기에 바로 그 페이지에서 회별 숫자를 받기 위해 title을 찾는다.
title_num=wd.find_elements_by_class_name('title')
#print(title_num[0].text) # 가장 첫 번째 회차의 이름을 받아옴.



wd.find_element_by_partial_link_text(title_num[0].text).click() # 가장 첫 번째 회차의 이름으로 해당 페이지에 접속
time.sleep(3)



#print(wd.current_url) # 현재 주소
recent_num_url = wd.current_url
# 가장 최신 회차의 주소를 알아내서 거기서 회차수만 받아옴.
no_result = re.search('no=(.+?)&', recent_num_url) # 주소에서 no = 이 부분이 회차 num인데 no=과 & 사이에 숫자만 받아옴
if no_result: #그에 대한 결과가 있다면
    found = no_result.group(1) # found에 저장함.
else : # 만약 주소에서 no이 가장 마지막에 위치한다면 결과값이 안나올 수 있기에 다시 한 번 else로 잡음
    no_result = re.search('no=(.+?)', recent_num_url)
    found = no_result.group(1)
    
int_found = int(found)# 가장 최근 회차를 int형으로 받아옴


# 최신 횟차 수 만큼 반복문을 돌려서 모든 회차의 댓글을 긁어옴,
for toon_part in range(int_found, 16, -1) :
    Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=747269&no=%s'% str(toon_part)
    wd.get(Webtoon_URL)
    wd.find_element_by_partial_link_text('전체댓글').click()        
    time.sleep(3)

    # 착한 댓글만 보게 하는 것 설정 제거
    wd.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
    time.sleep(0.4)

    while True : # 트루로 해두고
        #time.sleep(0.4)
        try:
            pages_num=wd.find_elements_by_class_name('u_cbox_num_page') # 현재 페이지에 뜨는 모든 페이지 num

            for i in range(len(pages_num)) : # 페이지 수만큼 진행
                time.sleep(0.4) #이거 안넣으면 첫페이지가 두번 크롤링됨
                #여기서 반복문으로 페이지마다 페이지 소스코드 받아외야 함.
                rcv_data = wd.page_source        
                soupData = BeautifulSoup(rcv_data, 'html.parser')

                # 현재 페이지 수 확인
                page_data=soupData.findAll('strong', attrs={'class': 'u_cbox_page'}) # 현재 눌려있는 페이지 번호만 갖는 class
                now_page=list(page_data[0].strings)[0] # page_data[0]는 하나뿐이라 무조건 0이고, 뒤에 []는 0이랑 1이 들어갈 수 있는데, 0이 해당하는 현제페이지수다.
                print(now_page)

                # 댓글 받아오기
                comment = soupData.findAll('span', attrs={'class': 'u_cbox_contents'})
                for comment_str in comment:
                    re = list(comment_str.strings)
                    result.append(re)

                #현재 페이지 번호(now_page)가 마지막 페이지 번호인지 확인하기 위해
                new_pages_num=wd.find_elements_by_class_name('u_cbox_num_page')# 창의 내용이 새로 고쳐져서 기존 객체를 계속 쓸 수 없기에 새 객체생성
                chk_lastnum =int(new_pages_num[len(new_pages_num)-1].text) # 현재페이지에 나와있는 페이지 번호 중 가장 마지막 페이지수

                if chk_lastnum > int(now_page) : #  마지막 페이지보다 현재페이지가 낮으면 계속 페이지를 넘기면 됨
                    new_pages_num[i+1].click() # 새로이 만든 객체로부터 click실행
                    time.sleep(0.3)


            temp_nextbtn=wd.find_element_by_class_name('u_cbox_next')        
            # 10,20,30 ... 페이지면 다음버튼을 누르고 아니라면 종료 & 마지막 페이지가 10의 배수일 수 있기에 다음버튼의 유무 확인
            if int(now_page) % 10 == 0 and temp_nextbtn.text != '' :
                wd.find_element_by_class_name('u_cbox_next').click()
                time.sleep(0.3)
                continue
            else :
                break   
            
        except UnexpectedAlertPresentException: # 데이터 전송에 실패했다는 문구 발생
                self.driver.switch_to.alert().accept()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165


30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68

In [3]:
comment_table = pd.DataFrame(result) # 데이터프레임을 만듦
comment_table.to_csv("./comment_last_to16.csv", mode='w', encoding='utf-8-sig')
#del result_mon[:]
print('FINISHED')

FINISHED


In [ ]:
# 얘는 16부터 0까지 ==> > 최종본(에러 없음)

In [9]:
#전독시 웹툰 페이지 : https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed
import re
result2 = []

wd = webdriver.Chrome('C:/Users/gjkh4/CRWL_김지현/chromedriver_win32/chromedriver.exe')
origin_URL = 'https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed'


# 먼저 전독시 웹툰 들어가서 가장 최근 만화 클릭하기
wd.get(origin_URL)
# 휴재가 아니라면 가장 최근 웹툰이 나오기에 바로 그 페이지에서 회별 숫자를 받기 위해 title을 찾는다.
title_num=wd.find_elements_by_class_name('title')
#print(title_num[0].text) # 가장 첫 번째 회차의 이름을 받아옴.



wd.find_element_by_partial_link_text(title_num[0].text).click() # 가장 첫 번째 회차의 이름으로 해당 페이지에 접속
time.sleep(3)



#print(wd.current_url) # 현재 주소
recent_num_url = wd.current_url
# 가장 최신 회차의 주소를 알아내서 거기서 회차수만 받아옴.
no_result = re.search('no=(.+?)&', recent_num_url) # 주소에서 no = 이 부분이 회차 num인데 no=과 & 사이에 숫자만 받아옴
if no_result: #그에 대한 결과가 있다면
    found = no_result.group(1) # found에 저장함.
else : # 만약 주소에서 no이 가장 마지막에 위치한다면 결과값이 안나올 수 있기에 다시 한 번 else로 잡음
    no_result = re.search('no=(.+?)', recent_num_url)
    found = no_result.group(1)
    
int_found = int(found)# 가장 최근 회차를 int형으로 받아옴


# 최신 횟차 수 만큼 반복문을 돌려서 모든 회차의 댓글을 긁어옴,
for toon_part in range(16, 4, -1) :
    Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=747269&no=%s'% str(toon_part)
    wd.get(Webtoon_URL)
    wd.find_element_by_partial_link_text('전체댓글').click()        
    time.sleep(3)

    # 착한 댓글만 보게 하는 것 설정 제거
    wd.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
    time.sleep(0.4)

    while True : # 트루로 해두고
        #time.sleep(0.4)
        try:
            pages_num=wd.find_elements_by_class_name('u_cbox_num_page') # 현재 페이지에 뜨는 모든 페이지 num
            time.sleep(0.4)
            for i in range(len(pages_num)) : # 페이지 수만큼 진행
                time.sleep(0.4) #이거 안넣으면 첫페이지가 두번 크롤링됨
                #여기서 반복문으로 페이지마다 페이지 소스코드 받아외야 함.
                rcv_data = wd.page_source        
                soupData = BeautifulSoup(rcv_data, 'html.parser')

                # 현재 페이지 수 확인
                page_data=soupData.findAll('strong', attrs={'class': 'u_cbox_page'}) # 현재 눌려있는 페이지 번호만 갖는 class
                now_page=list(page_data[0].strings)[0] # page_data[0]는 하나뿐이라 무조건 0이고, 뒤에 []는 0이랑 1이 들어갈 수 있는데, 0이 해당하는 현제페이지수다.
                print(now_page)

                # 댓글 받아오기
                comment = soupData.findAll('span', attrs={'class': 'u_cbox_contents'})
                for comment_str in comment:
                    re = list(comment_str.strings)
                    result2.append(re)

                #현재 페이지 번호(now_page)가 마지막 페이지 번호인지 확인하기 위해
                while True:
                    try:
                        new_pages_num=wd.find_elements_by_class_name('u_cbox_num_page')# 창의 내용이 새로 고쳐져서 기존 객체를 계속 쓸 수 없기에 새 객체생성
                        chk_lastnum =int(new_pages_num[len(new_pages_num)-1].text) # 현재페이지에 나와있는 페이지 번호 중 가장 마지막 페이지수

                        if chk_lastnum > int(now_page) : #  마지막 페이지보다 현재페이지가 낮으면 계속 페이지를 넘기면 됨
                            new_pages_num[i+1].click() # 새로이 만든 객체로부터 click실행
                            time.sleep(2)
                        break
                    except (NoSuchWindowException, NoSuchElementException):
                        pass
                
                new_pages_num=wd.find_elements_by_class_name('u_cbox_num_page')# 창의 내용이 새로 고쳐져서 기존 객체를 계속 쓸 수 없기에 새 객체생성
                chk_lastnum =int(new_pages_num[len(new_pages_num)-1].text) # 현재페이지에 나와있는 페이지 번호 중 가장 마지막 페이지수

                if chk_lastnum > int(now_page) : #  마지막 페이지보다 현재페이지가 낮으면 계속 페이지를 넘기면 됨
                    new_pages_num[i+1].click() # 새로이 만든 객체로부터 click실행
                    time.sleep(2)


            temp_nextbtn=wd.find_element_by_class_name('u_cbox_next')        
            # 10,20,30 ... 페이지면 다음버튼을 누르고 아니라면 종료 & 마지막 페이지가 10의 배수일 수 있기에 다음버튼의 유무 확인
            if int(now_page) % 10 == 0 and temp_nextbtn.text != '' :
                wd.find_element_by_class_name('u_cbox_next').click()
                time.sleep(2)
                continue
            else :
                break   
            
        except UnexpectedAlertPresentException: # 데이터 전송에 실패했다는 문구 발생
                self.driver.switch_to.alert().accept()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


In [10]:
comment_table2 = pd.DataFrame(result2) # 데이터프레임을 만듦
comment_table2.to_csv("./comment_16to4.csv", mode='w', encoding='utf-8-sig')
#del result_mon[:]
print('FINISHED')

FINISHED


In [ ]:
# 4부터 0까지 

In [11]:
#전독시 웹툰 페이지 : https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed
import re
result3 = []

wd = webdriver.Chrome('C:/Users/gjkh4/CRWL_김지현/chromedriver_win32/chromedriver.exe')
origin_URL = 'https://comic.naver.com/webtoon/list.nhn?titleId=747269&weekday=wed'



# 먼저 전독시 웹툰 들어가서 가장 최근 만화 클릭하기
wd.get(origin_URL)
# 휴재가 아니라면 가장 최근 웹툰이 나오기에 바로 그 페이지에서 회별 숫자를 받기 위해 title을 찾는다.
title_num=wd.find_elements_by_class_name('title')
#print(title_num[0].text) # 가장 첫 번째 회차의 이름을 받아옴.



wd.find_element_by_partial_link_text(title_num[0].text).click() # 가장 첫 번째 회차의 이름으로 해당 페이지에 접속
time.sleep(3)



#print(wd.current_url) # 현재 주소
recent_num_url = wd.current_url
# 가장 최신 회차의 주소를 알아내서 거기서 회차수만 받아옴.
no_result = re.search('no=(.+?)&', recent_num_url) # 주소에서 no = 이 부분이 회차 num인데 no=과 & 사이에 숫자만 받아옴
if no_result: #그에 대한 결과가 있다면
    found = no_result.group(1) # found에 저장함.
else : # 만약 주소에서 no이 가장 마지막에 위치한다면 결과값이 안나올 수 있기에 다시 한 번 else로 잡음
    no_result = re.search('no=(.+?)', recent_num_url)
    found = no_result.group(1)
    
int_found = int(found)# 가장 최근 회차를 int형으로 받아옴


# 최신 횟차 수 만큼 반복문을 돌려서 모든 회차의 댓글을 긁어옴,
for toon_part in range(4, 0, -1) :
    Webtoon_URL = 'https://comic.naver.com/comment/comment.nhn?titleId=747269&no=%s'% str(toon_part)
    wd.get(Webtoon_URL)
    wd.find_element_by_partial_link_text('전체댓글').click()        
    time.sleep(3)

    # 착한 댓글만 보게 하는 것 설정 제거
    wd.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
    time.sleep(0.4)
    wd.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
    time.sleep(0.4)

    while True : # 트루로 해두고
        #time.sleep(0.4)
        try:
            pages_num=wd.find_elements_by_class_name('u_cbox_num_page') # 현재 페이지에 뜨는 모든 페이지 num

            for i in range(len(pages_num)) : # 페이지 수만큼 진행
                time.sleep(0.4) #이거 안넣으면 첫페이지가 두번 크롤링됨
                #여기서 반복문으로 페이지마다 페이지 소스코드 받아외야 함.
                rcv_data = wd.page_source        
                soupData = BeautifulSoup(rcv_data, 'html.parser')

                # 현재 페이지 수 확인
                page_data=soupData.findAll('strong', attrs={'class': 'u_cbox_page'}) # 현재 눌려있는 페이지 번호만 갖는 class
                now_page=list(page_data[0].strings)[0] # page_data[0]는 하나뿐이라 무조건 0이고, 뒤에 []는 0이랑 1이 들어갈 수 있는데, 0이 해당하는 현제페이지수다.
                print(now_page)

                # 댓글 받아오기
                comment = soupData.findAll('span', attrs={'class': 'u_cbox_contents'})
                for comment_str in comment:
                    re = list(comment_str.strings)
                    result3.append(re)

                #현재 페이지 번호(now_page)가 마지막 페이지 번호인지 확인하기 위해
                new_pages_num=wd.find_elements_by_class_name('u_cbox_num_page')# 창의 내용이 새로 고쳐져서 기존 객체를 계속 쓸 수 없기에 새 객체생성
                chk_lastnum =int(new_pages_num[len(new_pages_num)-1].text) # 현재페이지에 나와있는 페이지 번호 중 가장 마지막 페이지수

                if chk_lastnum > int(now_page) : #  마지막 페이지보다 현재페이지가 낮으면 계속 페이지를 넘기면 됨
                    new_pages_num[i+1].click() # 새로이 만든 객체로부터 click실행
                    time.sleep(0.3)


            temp_nextbtn=wd.find_element_by_class_name('u_cbox_next')        
            # 10,20,30 ... 페이지면 다음버튼을 누르고 아니라면 종료 & 마지막 페이지가 10의 배수일 수 있기에 다음버튼의 유무 확인
            if int(now_page) % 10 == 0 and temp_nextbtn.text != '' :
                wd.find_element_by_class_name('u_cbox_next').click()
                time.sleep(0.3)
                continue
            else :
                break   
            
        except UnexpectedAlertPresentException: # 데이터 전송에 실패했다는 문구 발생
                self.driver.switch_to.alert().accept()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802


In [12]:
comment_table3 = pd.DataFrame(result3) # 데이터프레임을 만듦
comment_table3.to_csv("./comment_4to0.csv", mode='w', encoding='utf-8-sig')
#del result_mon[:]
print('FINISHED')

FINISHED
